# Deforestation Analysis for Algonquin Park

This notebook analyzes historical deforestation patterns in Algonquin Park using:
1. Hansen Global Forest Change dataset (2000-2023)
2. GLAD Global Land Cover and Land Use Change (GLCLUC2020) dataset

The analysis applies Canada's definition of forest and generates outputs to assess carbon sequestration feasibility.

In [ ]:
# Install required packages with specific versions to avoid compatibility issues
!pip install numpy pandas geopandas matplotlib scipy
!pip install earthengine-api
!pip install folium==0.14.0
!pip install geemap==0.20.6

# Restart runtime message
print("\n\nIMPORTANT: If you see package compatibility errors after this cell runs,")
print("please restart the runtime by clicking Runtime > Restart runtime in the menu above.")
print("Then run all cells from the beginning.")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configure paths for Google Drive
BASE_DIR = '/content/drive/MyDrive/wildlands_league'
STUDY_AREA_PATH = os.path.join(BASE_DIR, 'Algonquin_Park_Layers.gpkg')
OUTPUT_DIR = BASE_DIR

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

## Canada's Forest Definition

According to Natural Resources Canada, forests are defined as:

- Land spanning more than 0.5 hectares
- Tree height of at least 5 meters (at maturity in situ)
- Crown cover of more than 25%
- Width of at least 20 meters

We'll apply these criteria in our analysis.

In [ ]:
# Load the study area
try:
    study_area = gpd.read_file(STUDY_AREA_PATH)
    print(f"Loaded study area from {STUDY_AREA_PATH}")
    print(f"CRS: {study_area.crs}")
    print(f"Number of features: {len(study_area)}")
    
    # Convert to GeoJSON for Earth Engine
    study_area_json = study_area.to_crs("EPSG:4326").__geo_interface__
    
    # Create Earth Engine geometry
    study_area_ee = ee.FeatureCollection(study_area_json)
    
    # Display the study area
    fig, ax = plt.subplots(figsize=(10, 8))
    study_area.plot(ax=ax, color='none', edgecolor='black')
    ax.set_title('Algonquin Park Study Area')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    logging.error(f"Error loading study area: {str(e)}")
    raise

In [ ]:
# Load Hansen dataset
hansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# Apply Canada's forest definition (>25% canopy cover)
treecover2000 = hansen.select(['treecover2000'])
forest_cover = treecover2000.gte(25)

# Get forest loss year (1-23, representing 2001-2023)
loss_year = hansen.select(['lossyear'])

# Get forest gain (2000-2012)
gain = hansen.select(['gain'])

In [ ]:
# Now import geemap after Earth Engine is initialized
try:
    import geemap
    
    # Create a map
    map_obj = geemap.Map()
    
    # Add forest cover in 2000
    map_obj.add_layer(
        forest_cover.clip(study_area_ee),
        {'palette': ['green']},
        'Forest Cover 2000'
    )
    
    # Add forest loss by year
    map_obj.add_layer(
        loss_year.updateMask(loss_year.gt(0)).clip(study_area_ee),
        {
            'min': 1,
            'max': 23,
            'palette': ['red', 'orange', 'yellow', 'green']
        },
        'Forest Loss Year'
    )
    
    # Add study area boundary
    map_obj.add_layer(study_area_ee, {}, 'Study Area')
    
    # Center map on study area
    map_obj.center_object(study_area_ee, zoom=10)
    map_obj
except Exception as e:
    print(f"Error creating interactive map: {str(e)}")
    print("\nFallback to static visualization...")
    
    # Create a URL for the Earth Engine Data Catalog
    hansen_url = "https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2023_v1_11"
    print(f"\nYou can explore the Hansen dataset here: {hansen_url}")
    
    # Display a static message about the study area
    print(f"\nStudy area: Algonquin Park")
    print(f"Analysis will proceed with calculations even without visualization.")

In [ ]:
# Calculate area in hectares
pixel_area = ee.Image.pixelArea().divide(10000)  # Convert to hectares

# Calculate total forest area in 2000
forest_area_2000 = forest_cover.multiply(pixel_area).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=study_area_ee,
    scale=30,
    maxPixels=1e12
).get('treecover2000').getInfo()

print(f"Forest area in 2000: {forest_area_2000:.2f} hectares")

In [ ]:
# Calculate forest loss by year
loss_by_year = {}
for year in range(1, 24):  # 2001-2023
    # Create mask for this year's loss
    this_year_loss = loss_year.eq(year).And(forest_cover)
    
    # Calculate area lost this year
    area_lost = this_year_loss.multiply(pixel_area).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=study_area_ee,
        scale=30,
        maxPixels=1e12
    ).get('lossyear').getInfo()
    
    loss_by_year[year + 2000] = area_lost

# Calculate forest gain area (2000-2012)
gain_area = gain.And(forest_cover).multiply(pixel_area).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=study_area_ee,
    scale=30,
    maxPixels=1e12
).get('gain').getInfo()

print(f"Forest gain 2000-2012: {gain_area:.2f} hectares")

In [ ]:
# Create DataFrame for loss by year
loss_df = pd.DataFrame({
    'Year': list(loss_by_year.keys()),
    'Area_Lost_Ha': list(loss_by_year.values())
})

# Calculate cumulative loss and percentage
loss_df['Cumulative_Loss_Ha'] = loss_df['Area_Lost_Ha'].cumsum()
loss_df['Percent_of_2000_Forest'] = (loss_df['Area_Lost_Ha'] / forest_area_2000) * 100
loss_df['Cumulative_Percent'] = loss_df['Cumulative_Loss_Ha'] / forest_area_2000 * 100

# Calculate total loss and annual rate
total_loss = loss_df['Area_Lost_Ha'].sum()
annual_rate = total_loss / 23  # 23 years from 2001-2023
net_change = gain_area - total_loss

# Create summary DataFrame
summary_df = pd.DataFrame([
    {'Metric': 'Forest Area in 2000 (ha)', 'Value': forest_area_2000},
    {'Metric': 'Total Forest Loss 2001-2023 (ha)', 'Value': total_loss},
    {'Metric': 'Forest Gain 2000-2012 (ha)', 'Value': gain_area},
    {'Metric': 'Net Change (ha)', 'Value': net_change},
    {'Metric': 'Average Annual Loss (ha/year)', 'Value': annual_rate},
    {'Metric': 'Percent Forest Lost', 'Value': (total_loss / forest_area_2000) * 100}
])

# Display summary
print("\nDeforestation Analysis Summary:")
print(summary_df.to_string(index=False))

# Save to CSV
loss_path = os.path.join(OUTPUT_DIR, "deforestation_loss_by_year.csv")
summary_path = os.path.join(OUTPUT_DIR, "deforestation_summary.csv")

loss_df.to_csv(loss_path, index=False)
summary_df.to_csv(summary_path, index=False)

print(f"\nSaved deforestation results to:")
print(f"- {loss_path}")
print(f"- {summary_path}")

In [ ]:
# Plot deforestation trends
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Annual loss
ax1.bar(loss_df['Year'], loss_df['Area_Lost_Ha'], color='red', alpha=0.7)
ax1.set_xlabel('Year')
ax1.set_ylabel('Forest Loss (hectares)')
ax1.set_title('Annual Forest Loss in Algonquin Park (2001-2023)')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# Add trend line
z = np.polyfit(loss_df['Year'], loss_df['Area_Lost_Ha'], 1)
p = np.poly1d(z)
ax1.plot(loss_df['Year'], p(loss_df['Year']), "r--", alpha=0.8)

# Cumulative loss
ax2.plot(loss_df['Year'], loss_df['Cumulative_Loss_Ha'], 'b-', linewidth=2.5)
ax2.set_xlabel('Year')
ax2.set_ylabel('Cumulative Forest Loss (hectares)')
ax2.set_title('Cumulative Forest Loss in Algonquin Park (2001-2023)')
ax2.grid(linestyle='--', alpha=0.7)

# Add percentage on secondary y-axis
ax3 = ax2.twinx()
ax3.plot(loss_df['Year'], loss_df['Cumulative_Percent'], 'g--', linewidth=2)
ax3.set_ylabel('Percent of 2000 Forest Cover')

# Add legend
ax2.legend(['Cumulative Loss (ha)'], loc='upper left')
ax3.legend(['Percent of 2000 Forest'], loc='upper right')

plt.tight_layout()

# Save the figure
output_path = os.path.join(OUTPUT_DIR, "deforestation_trends.png")
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved deforestation trend plots to {output_path}")

## GLAD Land Cover Analysis

Now let's analyze the land cover using the GLAD GLCLUC2020 dataset

In [ ]:
# Load GLAD GLCLUC2020 dataset
try:
    glad_lc = ee.ImageCollection("projects/glad/GLCLU2020/V1")
    glad_lc_2020 = glad_lc.filterDate('2020-01-01', '2020-12-31').first()
    
    # Define land cover classes
    lc_classes = {
        1: 'Evergreen Forest',
        2: 'Deciduous Forest',
        3: 'Mixed Forest',
        4: 'Woody Wetland',
        5: 'Herbaceous Wetland',
        6: 'Grassland',
        7: 'Shrubland',
        8: 'Cropland',
        9: 'Bare Ground',
        10: 'Urban/Built-up',
        11: 'Water',
        12: 'Snow/Ice',
        13: 'Herbaceous Vegetation'
    }
    
    # Calculate area for each land cover class
    lc_areas = {}
    for lc_id, lc_name in lc_classes.items():
        # Create mask for this land cover class
        lc_mask = glad_lc_2020.eq(lc_id)
        
        # Calculate area
        area = lc_mask.multiply(pixel_area).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=study_area_ee,
            scale=30,
            maxPixels=1e12
        ).get('landcover').getInfo()
        
        lc_areas[lc_name] = area
    
    # Create DataFrame for land cover areas
    lc_df = pd.DataFrame({
        'Land_Cover_Class': list(lc_areas.keys()),
        'Area_Ha': list(lc_areas.values())
    })
    
    # Calculate percentage
    total_area = lc_df['Area_Ha'].sum()
    lc_df['Percent'] = (lc_df['Area_Ha'] / total_area) * 100
    
    # Sort by area
    lc_df = lc_df.sort_values('Area_Ha', ascending=False).reset_index(drop=True)
    
    # Display results
    print("\nLand Cover Analysis (GLAD GLCLUC2020):")
    print(lc_df.to_string(index=False))
    
    # Save to CSV
    lc_path = os.path.join(OUTPUT_DIR, "land_cover_analysis.csv")
    lc_df.to_csv(lc_path, index=False)
    print(f"\nSaved land cover analysis to {lc_path}")
    
    # Plot land cover distribution
    plt.figure(figsize=(12, 8))
    
    # Only plot classes with significant area (>1%)
    plot_df = lc_df[lc_df['Percent'] > 1].copy()
    
    # Create pie chart
    plt.pie(plot_df['Area_Ha'], labels=plot_df['Land_Cover_Class'], 
            autopct='%1.1f%%', startangle=90, shadow=True)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    plt.title('Land Cover Distribution in Algonquin Park (2020)')
    
    # Save the figure
    lc_plot_path = os.path.join(OUTPUT_DIR, "land_cover_distribution.png")
    plt.savefig(lc_plot_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Saved land cover distribution plot to {lc_plot_path}")
    
except Exception as e:
    logging.error(f"Error in GLAD land cover analysis: {str(e)}")
    print(f"\nError analyzing GLAD land cover: {str(e)}")
    print("This may be due to access restrictions or dataset availability.")

## Carbon Sequestration Feasibility Assessment

Based on the deforestation timing and patterns, we can assess the potential for carbon sequestration projects.

In [ ]:
# Calculate metrics relevant for carbon sequestration feasibility
try:
    # Calculate recent deforestation rate (last 5 years)
    recent_years = [2019, 2020, 2021, 2022, 2023]
    recent_loss = loss_df[loss_df['Year'].isin(recent_years)]['Area_Lost_Ha'].sum()
    recent_rate = recent_loss / 5  # 5 years
    
    # Calculate early period deforestation rate (2001-2005)
    early_years = [2001, 2002, 2003, 2004, 2005]
    early_loss = loss_df[loss_df['Year'].isin(early_years)]['Area_Lost_Ha'].sum()
    early_rate = early_loss / 5  # 5 years
    
    # Calculate trend (increasing or decreasing deforestation)
    trend_direction = "Increasing" if recent_rate > early_rate else "Decreasing"
    trend_percent = abs((recent_rate - early_rate) / early_rate * 100) if early_rate > 0 else 0
    
    # Create carbon sequestration feasibility assessment
    carbon_df = pd.DataFrame([
        {'Metric': 'Recent Deforestation Rate (2019-2023, ha/year)', 'Value': recent_rate},
        {'Metric': 'Early Deforestation Rate (2001-2005, ha/year)', 'Value': early_rate},
        {'Metric': 'Deforestation Trend', 'Value': f"{trend_direction} by {trend_percent:.1f}%"},
        {'Metric': 'Forest Remaining in 2023 (ha)', 'Value': forest_area_2000 - total_loss},
        {'Metric': 'Forest Remaining in 2023 (%)', 'Value': 100 - (total_loss / forest_area_2000 * 100)},
        {'Metric': 'Net Forest Change 2000-2023 (ha)', 'Value': net_change}
    ])
    
    # Display results
    print("\nCarbon Sequestration Feasibility Assessment:")
    print(carbon_df.to_string(index=False))
    
    # Save to CSV
    carbon_path = os.path.join(OUTPUT_DIR, "carbon_sequestration_assessment.csv")
    carbon_df.to_csv(carbon_path, index=False)
    print(f"\nSaved carbon sequestration assessment to {carbon_path}")
    
    # Provide a qualitative assessment
    print("\nQualitative Assessment:")
    
    if trend_direction == "Decreasing" and recent_rate < 0.5 * early_rate:
        print("✅ FAVORABLE: Deforestation rates have significantly decreased, suggesting effective forest management.")
        print("   This trend supports the feasibility of carbon sequestration projects.")
    elif trend_direction == "Decreasing":
        print("✓ MODERATELY FAVORABLE: Deforestation rates are decreasing, which is positive for carbon projects.")
        print("   Further reduction in deforestation rates would improve feasibility.")
    elif recent_rate > 1.5 * early_rate:
        print("❌ UNFAVORABLE: Deforestation rates have significantly increased, creating challenges for carbon projects.")
        print("   Addressing drivers of deforestation should be prioritized before carbon project implementation.")
    else:
        print("⚠️ NEUTRAL: Deforestation trends show mixed signals for carbon sequestration feasibility.")
        print("   Site-specific assessments and targeted interventions would be needed.")
    
    # Forest remaining assessment
    forest_remaining_percent = 100 - (total_loss / forest_area_2000 * 100)
    if forest_remaining_percent > 80:
        print("\n✅ HIGH FOREST INTEGRITY: Over 80% of forest cover remains intact since 2000.")
        print("   This high level of forest integrity is favorable for carbon projects.")
    elif forest_remaining_percent > 60:
        print("\n✓ MODERATE FOREST INTEGRITY: 60-80% of forest cover remains intact since 2000.")
        print("   This moderate level of forest integrity may support carbon projects with proper management.")
    else:
        print("\n⚠️ LOW FOREST INTEGRITY: Less than 60% of forest cover remains intact since 2000.")
        print("   This low level of forest integrity may limit carbon project potential.")
    
except Exception as e:
    logging.error(f"Error in carbon sequestration assessment: {str(e)}")
    print(f"\nError in carbon sequestration assessment: {str(e)}")